In [1]:
from google.colab import drive
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
drive.mount('/content/gdrive/')

root = r"/content/gdrive/My Drive/Puc/Projeto Final/Datasets/"
data_folder = "train/"
path = f"{root}/finetuning/devel_small"

!pip install bert-tensorflow keras_bert

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/
     |████████████████████████████████| 71kB 3.3MB/s 
  Created wheel for keras-bert: filename=keras_bert-0.73.0-cp36-none-any.whl size=37439 sha256=573c535565337dd75843b97cc008522840992cf008a2b88dde7bc1be6354296f
  Stored in directory: /root/.cache/pip/wheels/39/05/22/7d781a2f12a2db3e71e1e19c4498c98871c6335583cbd12178
  Created wheel for keras-transformer: filename=keras_transformer-0.29.0-cp36-none-any.whl size=13392 sha256=8661de63ec629efd67bebd77ad8b2939

In [2]:
import numpy as np
import pandas as pd
import os
import sys
import random
import keras
import tensorflow as tf
import json
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
import pickle

Using TensorFlow backend.
W0820 17:49:05.488395 140009068128128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
with open("/content/gdrive/My Drive/Puc/Projeto Final/Datasets/pickle/devel_small.dmp", "rb")as f:
    labels, tokens, ids = pickle.load(f)

def fix_it(df):
    # print(f"list values len {len(df)}")
    # print(f"shape 0 {df[0].shape}")
    df = pd.concat(df)
    # print(f"New df:{df.shape}")
    return df.fillna(0).values.astype('int32')

labels = fix_it(labels)
tokens = fix_it(tokens)
ids = fix_it(ids)
mask = (ids > 0).astype('int32')

In [4]:
from keras_bert.bert import get_model
from keras_bert.loader import load_trained_model_from_checkpoint
from keras.optimizers import Adam
adam = Adam(lr=2e-5,decay=0.01)
maxlen = labels.shape[1]
print('begin_build')

BERT_PRETRAINED_DIR = f"/content/gdrive/My Drive/Puc/Projeto Final/models/wwm_cased_L-24_H-1024_A-16"

config_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
checkpoint_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
model = load_trained_model_from_checkpoint(config_file, checkpoint_file, training=True,seq_len=maxlen)
model.summary()

begin_build


W0820 17:51:04.931889 140009068128128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0820 17:51:04.941629 140009068128128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0820 17:51:05.013199 140009068128128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0820 17:51:05.014871 140009068128128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0820 17:51:05.028814

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 256)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 256)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 256, 1024),  29691904    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 256, 1024)    2048        Input-Segment[0][0]              
__________________________________________________________________________________________________
Embedding-

In [0]:
model.layers[-1].output

<tf.Tensor 'NSP/Softmax:0' shape=(?, 2) dtype=float32>

In [0]:
from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda
from keras.models import Model
import keras.backend as K
import re
import codecs

sequence_output  = model.layers[-6].output
pool_output = Dense(maxlen, activation='relu',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),name = 'real_output')(sequence_output)
model3  = Model(inputs=model.input, outputs=pool_output)
model3.compile(loss='mean_squared_error', optimizer=adam)
model3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 256)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 256)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 256, 768), ( 22268928    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 256, 768)     1536        Input-Segment[0][0]              
__________________________________________________________________________________________________
Embedding-

In [0]:
callbacks=[
    EarlyStopping( patience = 4,monitor = 'val_loss', verbose = 1 ),
    ReduceLROnPlateau( patience = 2, monitor = 'val_loss', verbose = 1 ),
    ModelCheckpoint( f'{root}/keras/model_bet.h5', verbose = 1, save_best_only = True )
]

autoencoder_train = model.fit( t, t, epochs=n_epochs, batch_size=batch_size, shuffle=True, callbacks = callbacks, verbose = 1, validation_data=( tt, tt ) )